# 3.1 Consultas contra la base de datos local "ucm_nosql_pl"

## a.El Total de locales y terrazas por distrito y barrio: construye una consulta que permita obtener el total de locales y terrazas agrupados por cada distrito y barrio.


In [3]:
#------------Dependencies-----------#
import pandas as pd
import pymongo as mongo

In [9]:
def mostrar_resultado(result, n=5):
    """
    Convierte un cursor de MongoDB a DataFrame y muestra las primeras filas.

    Parámetros:
    - result: cursor devuelto por find() o aggregate()
    - n: número de filas a mostrar (por defecto 5)
    """
    df_resultado = pd.DataFrame(list(result))
    return df_resultado.head(n)


In [10]:
# Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = mongo.MongoClient('mongodb://localhost:27017/')
result = client['ucm_nosql_pl']['locales'].aggregate([
    {
        '$lookup': {
            'from': 'terrazas',
            'localField': 'id_local',
            'foreignField': 'id_local',
            'as': 'terrazas_asociadas'
        }
    }, {
        '$group': {
            '_id': {
                'distrito': '$desc_distrito_local',
                'barrio': '$desc_barrio_local'
            },
            'total_locales': {
                '$sum': 1
            },
            'locales_con_terraza': {
                '$sum': {
                    '$cond': [
                        {
                            '$gt': [
                                {
                                    '$size': '$terrazas_asociadas'
                                }, 0
                            ]
                        }, 1, 0
                    ]
                }
            }
        }
    }
])

mostrar_resultado(result,5)

,_id,total_locales,locales_con_terraza
0,"{'distrito': 'moncloa-aravaca', 'barrio': 'ciu...",898,22
1,"{'distrito': 'villa de vallecas', 'barrio': 'c...",3667,64
2,"{'distrito': 'ciudad lineal', 'barrio': 'ventas'}",2790,54
3,"{'distrito': 'moratalaz', 'barrio': 'fontarron'}",376,17
4,"{'distrito': 'arganzuela', 'barrio': 'legazpi'}",508,49


## b.Tipos de licencias y cantidad de licencias por cada tipo: crea una consulta para contar cuántas licencias hay de cada tipo en los datos.

In [13]:
from pymongo import MongoClient

# Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = MongoClient('mongodb://localhost:27017/')
result = client['ucm_nosql_pl']['licencias'].aggregate([
    {
        '$group': {
            '_id': {
                'id_tipo_licencia': '$id_tipo_licencia'
            },
            'total_licencias_x_tipo': {
                '$sum': 1
            }
        }
    }
])
mostrar_resultado(result,5)

,_id,total_licencias_x_tipo
0,{'id_tipo_licencia': 4},57028
1,{'id_tipo_licencia': 0},5956
2,{'id_tipo_licencia': 3},5813
3,{'id_tipo_licencia': 1},51249
4,{'id_tipo_licencia': 2},30783


## c.Listado de locales y terrazas con licencias “En trámite”: diseña una consulta que filtre y devuelva un listado detallado de locales y terrazas cuyo estado de licencia sea “En trámite”.

In [ ]:
 # Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = MongoClient('mongodb://localhost:27017/')
result = client['ucm_nosql_pl']['locales'].aggregate([
    {
        '$lookup': {
            'from': 'terrazas',
            'localField': 'id_local',
            'foreignField': 'id_local',
            'as': 'terrazas_asociadas'
        }
    }, {
        '$lookup': {
            'from': 'licencias',
            'localField': 'id_local',
            'foreignField': 'id_local',
            'as': 'licencias_x_local'
        }
    }, {
        '$match': {
            'licencias_x_local.desc_tipo_situacion_licencia': 'en tramitación'
        }
    }, {
        '$project': {
            'id_local': 1,
            'desc_distrito_local': 1,
            'desc_barrio_local': 1,
            'licencias_en_tramite': {
                '$filter': {
                    'input': '$licencias_x_local',
                    'as': 'lic',
                    'cond': {
                        '$eq': [
                            '$$lic.desc_tipo_situacion_licencia', 'en tramitación'
                        ]
                    }
                }
            }
        }
    }
])
mostrar_resultado(result,5)

![MongoDB_Carga_Exitosa](imagenes_capturas_mongo/query_C.png)


**Sin indices creados, las querys pueden ser muy largas si se trata de hacer uniones por referencia**

## d.Consulta por sección, división y epígrafe de la actividad comercial: crea una consulta para clasificar locales y terrazas según los campos sección, división y epígrafe.

In [14]:
# Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = MongoClient('mongodb://localhost:27017/')
result = client['ucm_nosql_pl']['locales'].aggregate([
    {
        '$lookup': {
            'from': 'terrazas',
            'localField': 'id_local',
            'foreignField': 'id_local',
            'as': 'terrazas_x_locales'
        }
    }, {
        '$match': {
            'actividades.0': {
                '$exists': True
            },
            'terrazas_x_locales.0': {
                '$exists': True
            }
        }
    }, {
        '$unwind': {
            'path': '$actividades'
        }
    }, {
        '$unwind': {
            'path': '$terrazas_x_locales'
        }
    }, {
        '$match': {
            '$and': [
                {
                    'actividades.desc_seccion': {
                        '$ne': None
                    }
                }, {
                    'actividades.desc_division': {
                        '$ne': None
                    }
                }, {
                    'actividades.desc_epigrafe': {
                        '$ne': None
                    }
                }
            ]
        }
    }, {
        '$group': {
            '_id': {
                'seccion': '$actividades.desc_seccion',
                'division': '$actividades.desc_division',
                'epigrafe': '$actividades.desc_epigrafe'
            },
            'total_locales_con_terraza': {
                '$sum': 1
            }
        }
    }
])
mostrar_resultado(result,5) #MUY PESADO SIN INDICES

,_id,total_locales_con_terraza
0,"{'seccion': 'actividades artísticas, recreativ...",2
1,"{'seccion': 'educación', 'division': 'educació...",1
2,"{'seccion': 'actividades artísticas, recreativ...",7
3,{'seccion': 'comercio al por mayor y al por me...,1
4,"{'seccion': 'hostelería', 'division': 'servici...",33


## e.Actividad económica más frecuente por barrio y distrito: diseña una consulta que identifique cuál es la actividad económica predominante en cada barrio y distrito.

In [15]:
# Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = MongoClient('mongodb://localhost:27017/')
result = client['ucm_nosql_pl']['locales'].aggregate([
    {
        '$unwind': {
            'path': '$actividades'
        }
    }, {
        '$group': {
            '_id': {
                'actividad': '$actividades.desc_division',
                'distrito': '$desc_distrito_local',
                'barrio': '$desc_barrio_local'
            },
            'numero_de_actividades': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            '_id.distrito': 1,
            '_id.barrio': 1,
            'numero_de_actividades': -1
        }
    }, {
        '$group': {
            '_id': {
                'distrito': '$_id.distrito',
                'barrio': '$_id.barrio'
            },
            'actividad_mas_frecuente': {
                '$first': '$_id.actividad'
            },
            'frecuencia': {
                '$first': '$numero_de_actividades'
            }
        }
    }
])
mostrar_resultado(result,5)

,_id,actividad_mas_frecuente,frecuencia
0,"{'distrito': 'carabanchel', 'barrio': 'san isi...",valor nulo en origen,888
1,"{'distrito': 'fuencarral-el pardo', 'barrio': ...",valor nulo en origen,6
2,"{'distrito': 'retiro', 'barrio': 'los jeronimos'}",servicios de comidas y bebidas,100
3,"{'distrito': 'arganzuela', 'barrio': 'atocha'}","comercio al por menor, excepto de vehículos de...",81
4,"{'distrito': 'centro', 'barrio': 'cortes'}",servicios de comidas y bebidas,448


## f.Actualización de horarios de apertura y cierre de ciertos locales: modifica los horarios de apertura y cierre de un conjunto seleccionado de locales según un criterio que tú determines.

![MongoDB_Carga_Exitosa](imagenes_capturas_mongo/Query_ultima.png)

##  Actualización de horarios de cierre en zonas turísticas

Como parte del mantenimiento y mejora del modelo de datos, se ha realizado una **actualización de los horarios de cierre (`hora_cierre1` y `hora_cierre2`)** de ciertos locales ubicados en zonas históricas y turísticas de la ciudad.

###  Criterio utilizado
Se han seleccionado los locales cuya propiedad `desc_distrito_local` corresponde a los siguientes distritos:

- **Centro**
- **Chamberí**
- **Retiro**

Estos distritos representan áreas céntricas con alta densidad de turismo, ocio nocturno y residencias, donde suelen aplicarse **normativas municipales de control del ruido y del descanso vecinal**.

###  Modificación aplicada
Se ha actualizado el horario de cierre de estos locales a las **01:00:00** en ambos campos de hora:

```json
{
  "hora_cierre1": "01:00:00",
  "hora_cierre2": "01:00:00"
}


## 3.2 Creación y uso de indices

![MongoDB_Creacion_Indices](imagenes_capturas_mongo/creacion_indices.png)

**Creación de indices**

![MongoDB_Prueba_Indice](imagenes_capturas_mongo/indice_usado.png)
**Prueba de uso de indice**

## Plan de ejecución de las consultas y diferencia de tiempos

## a.El Total de locales y terrazas por distrito y barrio: construye una consulta que permita obtener el total de locales y terrazas agrupados por cada distrito y barrio.


In [17]:
# Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = mongo.MongoClient('mongodb://localhost:27017/')
result = client['ucm_nosql_pl']['locales'].aggregate([
    {
        '$lookup': {
            'from': 'terrazas',
            'localField': 'id_local',
            'foreignField': 'id_local',
            'as': 'terrazas_asociadas'
        }
    }, {
        '$group': {
            '_id': {
                'distrito': '$desc_distrito_local',
                'barrio': '$desc_barrio_local'
            },
            'total_locales': {
                '$sum': 1
            },
            'locales_con_terraza': {
                '$sum': {
                    '$cond': [
                        {
                            '$gt': [
                                {
                                    '$size': '$terrazas_asociadas'
                                }, 0
                            ]
                        }, 1, 0
                    ]
                }
            }
        }
    }
])

mostrar_resultado(result,5)

,_id,total_locales,locales_con_terraza
0,"{'distrito': 'usera', 'barrio': 'almendrales'}",1058,19
1,"{'distrito': 'usera', 'barrio': 'moscardo'}",1492,40
2,"{'distrito': 'chamartin', 'barrio': 'castilla'}",756,43
3,"{'distrito': 'moncloa-aravaca', 'barrio': 'val...",101,2
4,"{'distrito': 'hortaleza', 'barrio': 'canillas'}",1964,61


## b.Tipos de licencias y cantidad de licencias por cada tipo: crea una consulta para contar cuántas licencias hay de cada tipo en los datos.

In [18]:
from pymongo import MongoClient

# Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = MongoClient('mongodb://localhost:27017/')
result = client['ucm_nosql_pl']['licencias'].aggregate([
    {
        '$group': {
            '_id': {
                'id_tipo_licencia': '$id_tipo_licencia'
            },
            'total_licencias_x_tipo': {
                '$sum': 1
            }
        }
    }
])
mostrar_resultado(result,5)

,_id,total_licencias_x_tipo
0,{'id_tipo_licencia': 3},5813
1,{'id_tipo_licencia': 1},51249
2,{'id_tipo_licencia': 2},30783
3,{'id_tipo_licencia': 4},57028
4,{'id_tipo_licencia': 0},5956


## c.Listado de locales y terrazas con licencias “En trámite”: diseña una consulta que filtre y devuelva un listado detallado de locales y terrazas cuyo estado de licencia sea “En trámite”.

## Creación de indices para mejorar el tiempo de respuesta de esta query que antes pasó mas de 5 minutos y no devolvía respuesta

![MongoDB_Prueba_Indice](imagenes_capturas_mongo/indice_usado.png)

In [21]:
 # Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = MongoClient('mongodb://localhost:27017/')
result = client['ucm_nosql_pl']['locales'].aggregate([
    {
        '$lookup': {
            'from': 'terrazas',
            'localField': 'id_local',
            'foreignField': 'id_local',
            'as': 'terrazas_asociadas'
        }
    }, {
        '$lookup': {
            'from': 'licencias',
            'localField': 'id_local',
            'foreignField': 'id_local',
            'as': 'licencias_x_local'
        }
    }, {
        '$match': {
            'licencias_x_local.desc_tipo_situacion_licencia': 'en tramitación'
        }
    }, {
        '$project': {
            'id_local': 1,
            'desc_distrito_local': 1,
            'desc_barrio_local': 1,
            'licencias_en_tramite': {
                '$filter': {
                    'input': '$licencias_x_local',
                    'as': 'lic',
                    'cond': {
                        '$eq': [
                            '$$lic.desc_tipo_situacion_licencia', 'en tramitación'
                        ]
                    }
                }
            }
        }
    }
])
mostrar_resultado(result,5)

,_id,id_local,desc_distrito_local,desc_barrio_local,licencias_en_tramite
0,685a81586586723066b49382,20000596,arganzuela,chopera,"[{'_id': 685a815d6586723066b87c64, 'id_local':..."
1,685a81586586723066b49385,20000709,arganzuela,acacias,"[{'_id': 685a815d6586723066b88458, 'id_local':..."
2,685a81586586723066b49387,20000729,arganzuela,delicias,"[{'_id': 685a815d6586723066b87da0, 'id_local':..."
3,685a81586586723066b49388,20000756,arganzuela,palos de la frontera,"[{'_id': 685a815d6586723066b881bf, 'id_local':..."
4,685a81586586723066b4938a,20000764,arganzuela,acacias,"[{'_id': 685a815d6586723066b88492, 'id_local':..."


## HA TARDADO SOLO 3 S 592 MS IMPRESIONANTE¡

## d.Consulta por sección, división y epígrafe de la actividad comercial: crea una consulta para clasificar locales y terrazas según los campos sección, división y epígrafe.

In [22]:
# Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = MongoClient('mongodb://localhost:27017/')
result = client['ucm_nosql_pl']['locales'].aggregate([
    {
        '$unwind': {
            'path': '$actividades'
        }
    }, {
        '$group': {
            '_id': {
                'actividad': '$actividades.desc_division',
                'distrito': '$desc_distrito_local',
                'barrio': '$desc_barrio_local'
            },
            'numero_de_actividades': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            '_id.distrito': 1,
            '_id.barrio': 1,
            'numero_de_actividades': -1
        }
    }, {
        '$group': {
            '_id': {
                'distrito': '$_id.distrito',
                'barrio': '$_id.barrio'
            },
            'actividad_mas_frecuente': {
                '$first': '$_id.actividad'
            },
            'frecuencia': {
                '$first': '$numero_de_actividades'
            }
        }
    }
])
mostrar_resultado(result,5)


,_id,actividad_mas_frecuente,frecuencia
0,"{'distrito': 'ciudad lineal', 'barrio': 'colina'}",valor nulo en origen,80
1,"{'distrito': 'puente de vallecas', 'barrio': '...",valor nulo en origen,675
2,"{'distrito': 'latina', 'barrio': 'campamento'}",valor nulo en origen,283
3,"{'distrito': 'chamartin', 'barrio': 'castilla'}",valor nulo en origen,210
4,"{'distrito': 'hortaleza', 'barrio': 'canillas'}",valor nulo en origen,632
